In [11]:
import numpy as np
from scipy.stats import rankdata
import pandas as pd

In [23]:
alternatif = ['Wiji','Sugiyem','Dul','Kuswara','Tugiyah']
kriteria = ['penghasilan','tanggungan','atap','dinding','komunikasi']
data = np.array([
            [4,4,3,5,3],
            [3,2,3,5,2],
            [4.5,5,2,2,3],
            [3.5,5,2,2,3],
            [3,2,3,5,2]
                ])


# Normalize the decision matrix
normalized_data = data / np.sqrt(np.sum(data**2,axis=0))
# normalized_data

print("DATA")
pd.DataFrame(data,columns=kriteria,index=alternatif)


DATA


,penghasilan,tanggungan,atap,dinding,komunikasi
Wiji,4.0,4.0,3.0,5.0,3.0
Sugiyem,3.0,2.0,3.0,5.0,2.0
Dul,4.5,5.0,2.0,2.0,3.0
Kuswara,3.5,5.0,2.0,2.0,3.0
Tugiyah,3.0,2.0,3.0,5.0,2.0


In [27]:
weights = np.array([5,4,4,5,3])
print('PEMBOBOTAN')

pd.DataFrame([weights],columns=kriteria,index=['bobot'])

PEMBOBOTAN


,penghasilan,tanggungan,atap,dinding,komunikasi
bobot,5,4,4,5,3


In [21]:
normalized_data

array([[0.49051147, 0.46499055, 0.50709255, 0.5488213 , 0.50709255],
       [0.3678836 , 0.23249528, 0.50709255, 0.5488213 , 0.3380617 ],
       [0.55182541, 0.58123819, 0.3380617 , 0.21952852, 0.50709255],
       [0.42919754, 0.58123819, 0.3380617 , 0.21952852, 0.50709255],
       [0.3678836 , 0.23249528, 0.50709255, 0.5488213 , 0.3380617 ]])

In [13]:

weighted = normalized_data * weights
weighted

array([[2.45255736, 1.85996222, 2.02837021, 2.7441065 , 1.52127766],
       [1.83941802, 0.92998111, 2.02837021, 2.7441065 , 1.01418511],
       [2.75912703, 2.32495277, 1.35224681, 1.0976426 , 1.52127766],
       [2.14598769, 2.32495277, 1.35224681, 1.0976426 , 1.52127766],
       [1.83941802, 0.92998111, 2.02837021, 2.7441065 , 1.01418511]])

In [14]:
#True jika  benefit dan False jika Cost
dec_matrix = np.array([False,True,True,True,True])

maximizing = np.where(dec_matrix,np.max(weighted,axis=0),
                      np.min(weighted,axis=0))
maximizing

array([1.83941802, 2.32495277, 2.02837021, 2.7441065 , 1.52127766])

In [15]:
minimizing = np.where(dec_matrix,np.min(weighted,axis=0),
                      np.max(weighted,axis=0))
minimizing

array([2.75912703, 0.92998111, 1.35224681, 1.0976426 , 1.01418511])

In [16]:
solusi_ideal_positif = np.sqrt(np.sum((maximizing-normalized_data)**2, 
                                   axis=1))


solusi_ideal_positif

array([3.66621639, 3.88295696, 3.86751128, 3.91004739, 3.88295696])

In [17]:
solusi_ideal_negatif = np.sqrt(np.sum((normalized_data - minimizing)**2, 
                                   axis=1))

solusi_ideal_negatif

array([2.57593986, 2.77076975, 2.65529672, 2.75807753, 2.77076975])

In [18]:
relative_closeness = solusi_ideal_negatif / (solusi_ideal_negatif+solusi_ideal_positif)

# Rank the alternatives
# ranked_alternatives = np.argsort(relative_closeness)
# ranked_alternatives.reshape([5,1] )
ranked_alternatives = rankdata(relative_closeness,method='ordinal')

relative_closeness,ranked_alternatives.reshape([data.shape[0],1])

(array([0.41266828, 0.41642374, 0.40707878, 0.41362115, 0.41642374]),
 array([[2],
        [4],
        [1],
        [3],
        [5]]))

In [19]:

import skcriteria as skc
from skcriteria.madm import simple
from skcriteria.madm import similarity
from skcriteria.pipeline import mkpipe
from skcriteria.preprocessing import invert_objectives,scalers

minmax = np.where(dec_matrix,max,min)
dm = skc.mkdm(data,minmax,weights,
              alternatives=alternatif,
              criteria=kriteria)


pipe = mkpipe(
    invert_objectives.NegateMinimize(),
    scalers.VectorScaler(target='matrix'),
    scalers.SumScaler(target='weights'),
    similarity.TOPSIS()
)

# dm.plot('kde')
#inverting it first
# dmt = invert_objectives.InvertMinimize().transform(dm)
# rankUsingWeightedSum = simple.WeightedSumModel().evaluate(dmt)
# rankUsingWeightedSum

dm = pipe.evaluate(dm)
dm

Alternatives,Wiji,Sugiyem,Dul,Kuswara,Tugiyah
Rank,1,2,4,3,2
